### Figure 2 : Ripple oscillations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from subjects import sns_boxplot_kw,stat_kw
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats
from statannotations.Annotator import Annotator


grpdata = subjects.GroupData()
group = ['NSD','SD']

# ripple
psd = grpdata.ripple_psd
examples = grpdata.ripple_examples
power= grpdata.ripple_zscore

ripple_autocorr = grpdata.ripple_autocorr
ripple_autocorr = ripple_autocorr[ripple_autocorr.lag_time>=-1]

rate = grpdata.ripple_rate
colors_ripple_rate = subjects.colors_sd(1.3)

total_dur = grpdata.ripple_total_duration
colors_rpl_dur = subjects.colors_sd(1.2)

peak_freq = grpdata.ripple_peak_frequency
peak_freq = peak_freq[(peak_freq['t']!='pre')&(peak_freq["t"]!="maze")]
colors_pf = subjects.colors_sd(1.3)

# pbe
pbe_rate = grpdata.pbe_rate
pbe_total_dur = grpdata.pbe_total_duration

# frate
frate_in_rpl = grpdata.frate_in_ripple
colors_frate = [
    [subjects.nsd.color(0.9), subjects.nsd.color(1.9)],
    [subjects.sd.color(1), subjects.sd.color(1.5)],
]

plt.clf()
fig = Fig(grid=(8, 11), fontsize=7)
filename = subjects.figpath_sd / "figure2"

### Peak frequency and power

In [ ]:
y_labels = ["Intraripple frequency (Hz)", "Ripple power (zscore)"]
for d, (df, y) in enumerate(zip([peak_freq, power], ["f", "zscore"])):
    ax = fig.subplot(fig.gs[0, 2 * d : 2 * d + 2])
    plot_kw = dict(data=df, x="t", y=y, hue="grp", hue_order=group, ax=ax)

    sns.boxplot(**plot_kw, width=0.5, **sns_boxplot_kw)
    ax.tick_params("x", labelrotation=30)
    ax.legend([], frameon=False)
    ax.set_ylabel(y_labels[d])
    ax.set_xlabel("")

    # stats
    orders = df.t.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw)

    annotator.apply_and_annotate()
    annotator.reset_configuration()


### Ripple autocorr
ax = fig.subplot(fig.gs[4])
sns.lineplot(
    data=ripple_autocorr,
    x="lag_time",
    y="auto_corr",
    hue="grp",
    ci=None,
    palette=subjects.colors_sd(1.4),
    lw=1,
)
ax.legend("", frameon=False)
ax.axis("off")


### Ripple and PBE rate

In [ ]:
y_labels = ["Ripple rate (Hz)", "PBE rate (Hz)"]
for d, (df, y) in enumerate(zip([rate, pbe_rate], ["rate", "rate"])):
    ax = fig.subplot(fig.gs[1, 2 * d : 2 * d + 2])
    plot_kw = dict(
        data=df,
        x="Zt",
        y=y,
        hue="grp",
        hue_order=group,
        ax=ax,
        dodge=True,
    )

    sns.stripplot(
        **plot_kw, palette=subjects.colors_sd(1), edgecolor="k", linewidth=0.3, size=3
    )
    sns.boxplot(**plot_kw, color="k", width=0.5, **sns_boxplot_kw)
    ax.tick_params("x", labelrotation=30)
    ax.legend([], frameon=False)
    ax.set_ylabel(y_labels[d])
    ax.set_xlabel("")

    # stats
    # Across groups
    orders = df.Zt.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    # Within groups
    pairs2 = []
    for g in group:
        pairs2.extend([((orders[_], g), (orders[_ + 1], g)) for _ in range(2)])
    annotator = Annotator(pairs=pairs2, **plot_kw, order=orders)
    annotator.configure(test="t-test_paired", **stat_kw)
    annotator.apply_and_annotate()
    annotator.reset_configuration()


### Firing inside/outside ripples

In [ ]:
k = 0
for i, w in enumerate(["inside", "outside"]):
    df_w = frate_in_rpl[frate_in_rpl["where"] == w]

    for i1, n in enumerate(["inter", "pyr"]):
        df = df_w[df_w["neuron_type"] == n].reset_index(drop=True)

        ax = plt.subplot(fig.gs[i + 2, 2 * i1 : 2 * i1 + 2])
        plot_kw = dict(data=df, x="zt", width=0.5, y="frate", hue="grp", ax=ax)

        sns.boxplot(**plot_kw, **sns_boxplot_kw)
        k = k + 1

        ax.set_ylabel("")
        ax.set_xlabel("")
        ax.legend("", frameon=False)
        ax.tick_params("x", labelrotation=30)
        ax.set_ylim(-20,600)

        if i1 == 0:
            ax.set_ylabel(f"Firing rate (z-scored)\n{w} ripples")
            fig.panel_label(ax, "C")

        # stats
        orders = df.zt.unique()
        pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
        annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
        annotator.configure(test="t-test_welch", **stat_kw)
        annotator.apply_and_annotate()
        annotator.reset_configuration()


### saving

In [ ]:
fig.savefig(filename)